In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.special import expit  
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [2]:
data = pd.read_csv('dataset/dataset_cleaned.csv')
data.head()

,Hiện tại bạn có đang làm dụng thuốc hay chất gì đó không ?,Hiện tại bạn có đang sử dụng thuốc liên quan đến thần kinh hay tâm thần gì không ?,"Trong nhà của bạn, có ai bị bệnh liên quan đến tâm thần, hoặc thần kinh không ?","Trong nhà của bạn, có ai dùng chất kích thích không ?","Đôi khi bạn có cảm thấy mình từng có các triệu chứng như trầm cảm , tự kỉ hoặc cảm xúc bất thường trong thời gian ngắn không(ví dụ: cảm thấy vui bất thường hoặc dễ bị xúc động chỉ qa một video hoặc overthink có động lực trong thời gian ngắn sau đó lại mất đi dù cố tìm lại nhưng cũng không được như lúc đầu)","Bạn thường xuyên làm mất, thất lạc hoặc làm hỏng những thứ cần thiết để hoàn thành công việc (ví dụ như điện thoại, kính mắt, giấy tờ, ví, chìa khóa, v.v.) như thế nào?","Bạn thường tránh né, không thích hoặc miễn cưỡng tham gia vào các nhiệm vụ đòi hỏi nỗ lực tinh thần hoặc suy nghĩ liên tục như thế nào?","Bạn thường xuyên bồn chồn, gõ tay, gõ chân hoặc ngọ nguậy trên ghế, Rung Chân Không như thế nào?","Bạn có thường xuyên cảm thấy mình 'đang di chuyển', hành động như thể bạn 'được điều khiển bởi một động cơ' không (ví dụ, bạn không thể hoặc cảm thấy không thoải mái khi phải đứng yên trong một khoảng thời gian dài, chẳng hạn như trong nhà hàng hoặc cuộc họp)?",Bạn thường rời khỏi chỗ ngồi bao nhiêu lần trong những tình huống mà người ta yêu cầu bạn phải ngồi yên (ví dụ: rời khỏi chỗ làm việc hoặc văn phòng)?,...,Bạn đã bao giờ cảm thấy như vậy hoặc ước rằng mình đã chết chưa?,Bạn có lo lắng về điều gì không hoặc có điều gì khiến bạn thực sự sợ hãi không?,Qua các câu hỏi ở trên bạn nghĩ mình có triệu chứng hoặc bản thân nhận thức được mình có một số triệu chứng đã xuất hiện rồi không?,Các triệu chứng có xuất hiện ở ít nhất hai hoặc nhiều bối cảnh (ví dụ: ở nhà và trường học) không?,Bạn thường hay để quên các vật dụng hay sử dụng hoặc đồ vật vừa mới cầm với tần suất như thế nào ?,Bạn có thường hay có cảm giác động lực ảo làm việc với cường độ cao trong thời gian dài với 1 thứ gì đó trong 1 khoảng thời gian rồi lại bỏ ngang không,"Trong 1h tiếng hoặc thời gian xem 1 chương trình tần suất bạn thay đổi tư thế hoặc bấm điện thoại ,đọc sách làm việc khác là bao nhiêu lần và nó chiếm bao nhiêu % bộ phim",Bạn có thường hay làm việc riêng khi đi học đi làm không ?,Bạn đã bao giờ tự áp đặt bản thân phải làm việc quá nhiều dẫn đến mất phương hướng chán nản mất hết động lực chưa?,"Bạn đã bao giờ ? Đôi khi nằm im một chỗ không làm gì cả trong nhiều giờ liền ?\r\nNhiều ý tưởng nảy ra trong đầu suy nghĩ táo bạo, não bộ nhạy bén hiểu vấn đề nhanh phân tích tốt ?"
0,0,0,0,1,1,1,2,3,2,1,...,0,1,0,1,1,2,0,1,0,0
1,0,0,0,0,1,1,1,1,2,1,...,0,1,1,1,1,2,0,1,1,1
2,0,0,0,0,0,2,2,3,3,2,...,1,1,1,1,2,3,3,1,0,1
3,0,0,0,0,0,2,2,0,2,0,...,0,1,0,0,2,2,0,1,1,1
4,0,1,0,0,1,2,2,0,0,0,...,0,1,1,0,3,1,0,1,0,0


In [3]:
symptom_groups = {
    'impulsive_behaviors': {
        'columns': [
            'Bạn thường buột miệng trả lời trước khi câu hỏi được nói xong bao nhiêu lần (ví dụ, khi hoàn thành câu của người khác hoặc không thể chờ đến lượt mình trong một cuộc trò chuyện)  Hoặc có bao giờ hành động nóng vội dẫn tới tình trạng nghĩ tới đâu làm tới đó không?',
            'Bạn thường xuyên ngắt lời hoặc xen vào chuyện của người khác như xen vào cuộc trò chuyện hoặc chiếm mất việc của người khác như thế nào ?',
            'Bạn thường xuyên khó chịu khi người khác làm một việc gì đó quá lâu, và không hoàn thành trước mặt bạn không ?'
        ],
        'threshold': 6,
        'max_score': 9
    },
    'restlessness': {
        'columns': [
            'Bạn thường xuyên bồn chồn, gõ tay, gõ chân hoặc ngọ nguậy trên ghế, Rung Chân Không như thế nào?',
            'Bạn có thường xuyên cảm thấy mình \'đang di chuyển\', hành động như thể bạn \'được điều khiển bởi một động cơ\' không (ví dụ, bạn không thể hoặc cảm thấy không thoải mái khi phải đứng yên trong một khoảng thời gian dài, chẳng hạn như trong nhà hàng hoặc cuộc họp)?',
            'Bạn thường rời khỏi chỗ ngồi bao nhiêu lần trong những tình huống mà người ta yêu cầu bạn phải ngồi yên (ví dụ: rời khỏi chỗ làm việc hoặc văn phòng)?',
            'Bạn có thường xuyên cảm thấy bồn chồn - muốn ra ngoài và làm gì đó không?',
            'Bạn có thường hay làm việc riêng khi đi học đi làm không ?'
        ],
        'threshold': 9,
        'max_score': 13
    },
    'procrastination': {
        'columns': [
            'Bạn thường tránh né, không thích hoặc miễn cưỡng tham gia vào các nhiệm vụ đòi hỏi nỗ lực tinh thần hoặc suy nghĩ liên tục như thế nào?',
            'Bạn có thường hay có cảm giác động lực ảo làm việc với cường độ cao trong thời gian dài với 1 thứ gì đó trong 1 khoảng thời gian rồi lại bỏ ngang không',
            'Bạn đã bao giờ tự áp đặt bản thân phải làm việc quá nhiều dẫn đến mất phương hướng chán nản mất hết động lực chưa?'
        ],
        'threshold': 5,
        'max_score': 7
    },
    'losing_items': {
        'columns': [
            'Bạn thường xuyên làm mất, thất lạc hoặc làm hỏng những thứ cần thiết để hoàn thành công việc (ví dụ như điện thoại, kính mắt, giấy tờ, ví, chìa khóa, v.v.) như thế nào?',
            'Bạn thường hay để quên các vật dụng hay sử dụng hoặc đồ vật vừa mới cầm với tần suất như thế nào ?'
        ],
        'threshold': 4,
        'max_score': 6
    },
    'unable_to_sit_still': {
        'columns': [
            'Bạn thường rời khỏi chỗ ngồi bao nhiêu lần trong những tình huống mà người ta yêu cầu bạn phải ngồi yên (ví dụ: rời khỏi chỗ làm việc hoặc văn phòng)?',
            'Trong 1h tiếng hoặc thời gian xem 1 chương trình tần suất bạn thay đổi tư thế hoặc bấm điện thoại ,đọc sách làm việc khác là bao nhiêu lần và nó chiếm bao nhiêu % bộ phim'
        ],
        'threshold': 4,
        'max_score': 6
    },
    'emotional_instability': {
        'columns': [
            'Đôi khi bạn có cảm thấy mình từng có các triệu chứng như trầm cảm , tự kỉ hoặc cảm xúc bất thường trong thời gian ngắn không(ví dụ: cảm thấy vui bất thường hoặc dễ bị xúc động chỉ qa một video hoặc overthink có động lực trong thời gian ngắn sau đó lại mất đi dù cố tìm lại nhưng cũng không được như lúc đầu)',
            'Bạn đã bao giờ cảm thấy như vậy hoặc ước rằng mình đã chết chưa?',
            'Bạn có lo lắng về điều gì không hoặc có điều gì khiến bạn thực sự sợ hãi không?'
        ],
        'threshold': 2,
        'max_score': 3
    }
}

In [4]:
for group, info in symptom_groups.items():
    data[group] = data[info['columns']].sum(axis=1)
    data[f'{group}_flag'] = (data[group] > info['threshold']).astype(int)

    
data['ADHD'] = ((data[[f'{group}_flag' for group in symptom_groups]] == 1).sum(axis=1) >= 3).astype(int)

In [5]:
X = data[[group for group in symptom_groups]]
y = data['ADHD']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

d:\Dev\MasterUIT\SourceTest\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
history = model.fit(X_train_scaled, y_train, epochs=1000, batch_size=32, validation_split=0.2, verbose=0)
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Độ chính xác trên tập kiểm tra: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8750 - loss: 0.8345
Độ chính xác trên tập kiểm tra: 87.50%


In [9]:
X_all_scaled = scaler.transform(X)
predictions = model.predict(X_all_scaled)
adhd_predictions = (predictions > 0.5).astype(int)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


In [11]:
print(f"Số lượng bản ghi có khả năng bị ADHD: {np.sum(adhd_predictions)}")


Số lượng bản ghi có khả năng bị ADHD: 10


In [13]:
for i, pred in enumerate(adhd_predictions):
    if pred[0] == 1:
        print(f"\nBản ghi {i + 1} có khả năng bị ADHD:")
        for group, info in symptom_groups.items():
            score = X[group].iloc[i]
            print(f"  - {group}: {score}/{info['max_score']} điểm " + 
                  f"({'Vượt ngưỡng' if score > info['threshold'] else 'Bình thường'})")

# 13. Thống kê bổ sung
print("\nThống kê bổ sung:")
for group in symptom_groups:
    print(f"{group}:")
    print(f"  Trung bình: {X[group].mean():.2f}")
    print(f"  Độ lệch chuẩn: {X[group].std():.2f}")
    print(f"  Min: {X[group].min()}")
    print(f"  Max: {X[group].max()}")
    print(f"  Số lượng vượt ngưỡng: {data[f'{group}_flag'].sum()}")
    print()

print(f"Tổng số bản ghi: {len(data)}")
print(f"Số lượng được dự đoán có ADHD: {np.sum(adhd_predictions)}")
print(f"Tỷ lệ ADHD: {np.sum(adhd_predictions) / len(data) * 100:.2f}%")


Bản ghi 3 có khả năng bị ADHD:
  - impulsive_behaviors: 8/9 điểm (Vượt ngưỡng)
  - restlessness: 11/13 điểm (Vượt ngưỡng)
  - procrastination: 5/7 điểm (Bình thường)
  - losing_items: 4/6 điểm (Bình thường)
  - unable_to_sit_still: 5/6 điểm (Vượt ngưỡng)
  - emotional_instability: 2/3 điểm (Bình thường)

Bản ghi 7 có khả năng bị ADHD:
  - impulsive_behaviors: 5/9 điểm (Bình thường)
  - restlessness: 11/13 điểm (Vượt ngưỡng)
  - procrastination: 5/7 điểm (Bình thường)
  - losing_items: 5/6 điểm (Vượt ngưỡng)
  - unable_to_sit_still: 5/6 điểm (Vượt ngưỡng)
  - emotional_instability: 1/3 điểm (Bình thường)

Bản ghi 9 có khả năng bị ADHD:
  - impulsive_behaviors: 8/9 điểm (Vượt ngưỡng)
  - restlessness: 10/13 điểm (Vượt ngưỡng)
  - procrastination: 5/7 điểm (Bình thường)
  - losing_items: 4/6 điểm (Bình thường)
  - unable_to_sit_still: 5/6 điểm (Vượt ngưỡng)
  - emotional_instability: 3/3 điểm (Vượt ngưỡng)

Bản ghi 13 có khả năng bị ADHD:
  - impulsive_behaviors: 5/9 điểm (Bình thường)
 

In [17]:
X_all_scaled = scaler.transform(X)
adhd_predictions = model.predict(X_all_scaled)
confidence_scores = expit(abs(adhd_predictions))  

result_df = pd.DataFrame()

# Thêm các cột từ dữ liệu gốc
for group in symptom_groups:
    result_df[group] = X[group]

# Thêm cột dự đoán ADHD và độ tin tưởng
result_df['ADHD_Prediction'] = adhd_predictions
result_df['Confidence_Score'] = confidence_scores

# Thêm cột dự đoán rõ ràng
result_df['Predicted_Status'] = ['Có khả năng bị ADHD' if pred == 1 else 'Không có khả năng bị ADHD' for pred in adhd_predictions]

# Thêm cột nhãn ADHD thực tế (nếu có)
if 'ADHD' in data.columns:
    result_df['ADHD_Actual'] = data['ADHD']

# Thêm cột chỉ ra liệu dự đoán có đúng không (nếu có dữ liệu thực tế)
if 'ADHD' in data.columns:
    result_df['Prediction_Correct'] = (result_df['ADHD_Prediction'] == result_df['ADHD_Actual']).astype(int)

# Thêm các cột cờ cho từng nhóm triệu chứng
for group in symptom_groups:
    result_df[f'{group}_flag'] = data[f'{group}_flag']

# Thêm cột kết quả đánh giá cuối cùng
def get_final_assessment(prediction, confidence):
    if prediction == 1:
        if confidence >= 0.8:
            return "Rất có khả năng bị ADHD"
        elif confidence >= 0.6:
            return "Có khả năng bị ADHD"
        else:
            return "Có dấu hiệu ADHD, cần đánh giá thêm"
    else:
        if confidence >= 0.8:
            return "Rất có khả năng không bị ADHD"
        elif confidence >= 0.6:
            return "Có khả năng không bị ADHD"
        else:
            return "Không có dấu hiệu rõ ràng, cần đánh giá thêm"

result_df['Final_Assessment'] = [get_final_assessment(pred, conf) for pred, conf in zip(result_df['ADHD_Prediction'], result_df['Confidence_Score'])]

# 14. Lưu kết quả vào file CSV
output_file = 'adhd_prediction_results.csv'
result_df.to_csv(output_file, index=False)
print(f"\nKết quả đã được lưu vào file: {output_file}")

# 15. Hiển thị một số thống kê tổng quan
print("\nThống kê tổng quan:")
print(f"Tổng số bản ghi: {len(result_df)}")
print(f"Số lượng được dự đoán có ADHD: {result_df['ADHD_Prediction'].sum()}")
print(f"Tỷ lệ dự đoán ADHD: {result_df['ADHD_Prediction'].mean() * 100:.2f}%")
print(f"Độ tin tưởng trung bình: {result_df['Confidence_Score'].mean() * 100:.2f}%")

if 'ADHD_Actual' in result_df.columns:
    print(f"Số lượng thực tế có ADHD: {result_df['ADHD_Actual'].sum()}")
    print(f"Tỷ lệ thực tế ADHD: {result_df['ADHD_Actual'].mean() * 100:.2f}%")
    print(f"Độ chính xác tổng thể: {result_df['Prediction_Correct'].mean() * 100:.2f}%")

print("\nMẫu dữ liệu (5 bản ghi đầu tiên):")
print(result_df.head())

# 16. Phân tích độ tin tưởng
print("\nPhân tích độ tin tưởng:")
confidence_bins = [0, 0.6, 0.7, 0.8, 0.9, 1.0]
confidence_labels = ['<60%', '60-70%', '70-80%', '80-90%', '90-100%']
result_df['Confidence_Range'] = pd.cut(result_df['Confidence_Score'], bins=confidence_bins, labels=confidence_labels, include_lowest=True)

confidence_analysis = result_df.groupby('Confidence_Range')['ADHD_Prediction'].agg(['count', 'mean'])
confidence_analysis.columns = ['Số lượng', 'Tỷ lệ ADHD']
confidence_analysis['Tỷ lệ ADHD'] *= 100
print(confidence_analysis)

# 17. Lưu phân tích độ tin tưởng vào file CSV riêng
confidence_analysis.to_csv('adhd_confidence_analysis.csv')
print("\nPhân tích độ tin tưởng đã được lưu vào file: adhd_confidence_analysis.csv")

# 18. Phân tích kết quả đánh giá cuối cùng
print("\nPhân tích kết quả đánh giá cuối cùng:")
final_assessment_analysis = result_df['Final_Assessment'].value_counts()
print(final_assessment_analysis)

# 19. Lưu phân tích kết quả đánh giá cuối cùng vào file CSV riêng
final_assessment_analysis.to_csv('adhd_final_assessment_analysis.csv')
print("\nPhân tích kết quả đánh giá cuối cùng đã được lưu vào file: adhd_final_assessment_analysis.csv")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step

Kết quả đã được lưu vào file: adhd_prediction_results.csv

Thống kê tổng quan:
Tổng số bản ghi: 37
Số lượng được dự đoán có ADHD: 9.660955429077148
Tỷ lệ dự đoán ADHD: 26.11%
Độ tin tưởng trung bình: 56.06%
Số lượng thực tế có ADHD: 9
Tỷ lệ thực tế ADHD: 24.32%
Độ chính xác tổng thể: 0.00%

Mẫu dữ liệu (5 bản ghi đầu tiên):
   impulsive_behaviors  restlessness  procrastination  losing_items  \
0                    4             7                4             2   
1                    4             7                4             2   
2                    8            11                5             4   
3                    4             4                5             4   
4                    2             3                3             5   

   unable_to_sit_still  emotional_instability  ADHD_Prediction  \
0                    1                      2     6.719871e-08   
1                    1                      2     6.719871e-08   
2        

C:\Users\nguye\AppData\Local\Temp\ipykernel_23748\2634658555.py:75: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  confidence_analysis = result_df.groupby('Confidence_Range')['ADHD_Prediction'].agg(['count', 'mean'])
